In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

2.0.0-alpha0
2.2.4-tf


In [2]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

In [3]:
inputs = keras.Input(shape=(72,))
h1 = keras.layers.Dense(32,activation='relu')(inputs)
h2 = keras.layers.Dense(32,activation='relu')(h1)
pred = keras.layers.Dense(10,activation='relu')(h2)
model = keras.Model(inputs = inputs,outputs = pred)

model.compile(optimizer = keras.optimizers.Adam(learning_rate=1e-4),
             loss = keras.losses.categorical_crossentropy,
             metrics = [keras.metrics.categorical_accuracy])
model.fit(train_x,train_y,batch_size = 32,epochs = 5)

Epoch 1/5
1000/1000 [==============================] - 0s 171us/sample - loss: 41.6619 - categorical_accuracy: 0.0910
Epoch 2/5
1000/1000 [==============================] - 0s 32us/sample - loss: 38.4217 - categorical_accuracy: 0.0930
Epoch 3/5
1000/1000 [==============================] - 0s 32us/sample - loss: 35.9084 - categorical_accuracy: 0.0940
Epoch 4/5
1000/1000 [==============================] - 0s 31us/sample - loss: 34.0455 - categorical_accuracy: 0.0960
Epoch 5/5
1000/1000 [==============================] - 0s 32us/sample - loss: 32.6138 - categorical_accuracy: 0.1010


In [8]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = keras.layers.Dense(32, activation='relu')
        self.layer2 = keras.layers.Dense(num_classes, activation='softmax')
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 138us/sample - loss: 13.3328 - accuracy: 0.1150
Epoch 2/5
1000/1000 [==============================] - 0s 51us/sample - loss: 18.2281 - accuracy: 0.0960
Epoch 3/5
1000/1000 [==============================] - 0s 55us/sample - loss: 23.5314 - accuracy: 0.1000
Epoch 4/5
1000/1000 [==============================] - 0s 59us/sample - loss: 28.1745 - accuracy: 0.1060
Epoch 5/5
1000/1000 [==============================] - 0s 51us/sample - loss: 32.4452 - accuracy: 0.1020


In [10]:
class MyLayer(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                   initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

model = tf.keras.Sequential(
[
    MyLayer(10),
    keras.layers.Activation('softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 109us/sample - loss: 11.5177 - accuracy: 0.0810
Epoch 2/5
1000/1000 [==============================] - 0s 51us/sample - loss: 11.5147 - accuracy: 0.0810
Epoch 3/5
1000/1000 [==============================] - 0s 52us/sample - loss: 11.5079 - accuracy: 0.0830
Epoch 4/5
1000/1000 [==============================] - 0s 49us/sample - loss: 11.5025 - accuracy: 0.0860
Epoch 5/5
1000/1000 [==============================] - 0s 47us/sample - loss: 11.4999 - accuracy: 0.0850


In [ ]:
callbacks = [
    #如果两轮loss没有下降，停止训练
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='loss'),
    #保存tensorboard日志,目录
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks,validation_data=(val_x,val_y))

In [22]:
model = tf.keras.Sequential([
keras.layers.Dense(64, activation='relu'),
keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.save_weights('./weights/model')
model.load_weights('./weights/model')
model.save_weights('./model.h5')
model.load_weights('./model.h5')

In [30]:
# 序列化成json
import json
import pprint
json_str = model.to_json()
pprint.pprint(json.loads(json_str))
fresh_model = tf.keras.models.model_from_json(json_str)
# 保持为yaml格式  #需要提前安装pyyaml

yaml_str = model.to_yaml()
print(yaml_str)
fresh_model = tf.keras.models.model_from_yaml(yaml_str)

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': None,
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense_17',
                                   'trainable': True,
                                   'units': 64,

ImportError: Requires yaml module installed (`pip install pyyaml`).

In [4]:
model = tf.keras.Sequential([
  keras.layers.Dense(10, activation='softmax', input_shape=(72,)),
  keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)
model.save('all_model.h5')
model = tf.keras.models.load_model('all_model.h5')

In [34]:
model = tf.keras.Sequential([keras.layers.Dense(10,activation='softmax'),
                          keras.layers.Dense(10,activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(model)

W0705 14:19:05.695875 17244 estimator.py:1799] Using temporary folder as model directory: C:\Users\Xiaoi\AppData\Local\Temp\tmpbu5xmhaj


In [36]:
# 模型构造
inputs = keras.Input(shape=(784,), name='mnist_input')
h1 = keras.layers.Dense(64, activation='relu')(inputs)
h1 = keras.layers.Dense(64, activation='relu')(h1)
outputs = keras.layers.Dense(10, activation='softmax')(h1)
model = keras.Model(inputs, outputs)
model.summary()

model.compile(optimizer=keras.optimizers.RMSprop(),
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=[keras.metrics.SparseCategoricalAccuracy()])

# 载入数据
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') /255
x_test = x_test.reshape(10000, 784).astype('float32') /255

x_val = x_train[-10000:]
y_val = y_train[-10000:]

x_train = x_train[:-10000]
y_train = y_train[:-10000]

# 训练模型
history = model.fit(x_train, y_train, batch_size=64, epochs=3,
         validation_data=(x_val, y_val))
print('history:')
print(history.history)

result = model.evaluate(x_test, y_test, batch_size=128)
print('evaluate:')
print(result)
pred = model.predict(x_test[:2])
print('predict:')
print(pred)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mnist_input (InputLayer)     [(None, 784)]             0         
_________________________________________________________________
dense_23 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_24 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_25 (Dense)             (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
11493376/11490434 [==============================] - 1s 0us/step
Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 2s 41us/sample - loss: 0.3445 - sparse_categorical_accuracy: 0.8987 - val_loss: 0